# General tool for creating Inverse Kinematics setup xml file

> Alex Woodall<br>
> Auckland Bioengineering Institute<br>
> Auckland, New Zealand

This notebook will create an inverse kinematics setup xml file using the OpenSim API in Python.

_Note:_ This is written in Python 3.7.4 with OpenSim 4.0. Make sure you have read the [Python 3 OpenSim API Setup](OpenSim4.0Python3API.ipynb) and testing that the OpenSim API is working with Python 3.

__Assuming that the motion capture data was pre-processed in Nexus to remove gaps and duplicate markers.__

This script is better used as a function call where the input variables have been created prior to calling.

Start by importing the necessary libraries

In [17]:
import numpy as np
import opensim as osim
from xml.dom import minidom
import sys

sys.path.insert(1, r'Functions')  # add to path
from xml_shorten import xml_shorten

## Requirements

To generate the inverse kinematics setup xml file, we will need six items:
- __ik_filename__: the full filename for the template inverse kinematics setup xml file
- __trial__: the trial name, e.g., "12_Mar_ss_12ms_01"
- __model__: the model name, e.g., "AB08"
- __directory__: the output directory name
- __time_range__: the start and end times of the trial
- __marker_names__: the marker names which we used during the motion capture trial

In [9]:
# Establish input variables example
ik_filename = 'C:\\Users\\alexw\\Dropbox\\ABI\\Level_8_Lab\\OpenSim Tools\\ProcessingTrialDataFromVicon\\xmlTemplates\\IKSetup.xml'
trial = '_12Mar_ss_12ms_01'
model = 'AB08'
directory = 'C:\\Users\\alexw\\Dropbox\\ABI\\Level_8_Lab\\OpenSim Tools\\ProcessingTrialDataFromVicon\\Output'
time_range = [15.3, 30.455]
marker_names = ['LASI', 'RASI', 'LPSI', 'RPSI', 'LTH1', 'LTH2', 'LTH3', 'LTH4', 'LMFC', 'LLFC', 'LTB1', 'LTB2', 'LTB3', 'LTB4', 
                'LMMAL', 'LLMAL', 'LCAL', 'LToe', 'LMT1', 'LMT5', 'RTH1', 'RTH2', 'RTH3', 'RTH4', 'RMFC', 'RLFC', 'RTB1', 'RTB2', 
                'RTB3', 'RTB4', 'RMMAL', 'RLMAL', 'RCAL', 'RToe', 'RMT1', 'RMT5', 'C7', 'T10', 'LACR1', 'RACR1', 'CLAV', 'RBack']

## Establish output and time variables

In [10]:
# Create an instance of the inverse kinematics tool
IK_tool = osim.InverseKinematicsTool(ik_filename)

# Set the name of the tool
IK_tool.setName(model)

# Set the time range, NOTE: Must be a double (np.float64)
IK_tool.setStartTime(np.float64(time_range[0]))
IK_tool.setEndTime(np.float64(time_range[-1]))

# Set the marker file
marker_file_name = trial + ".trc"
IK_tool.setMarkerDataFileName(marker_file_name)

# Set the coordinate file
coordinate_file_name = ''
IK_tool.setCoordinateFileName(coordinate_file_name)

# Set the output motion file
output_file_name = trial + "IKResults.mot"
IK_tool.setOutputMotionFileName(output_file_name)

# Set the input and results directory
IK_tool.setInputsDir(".\\")
IK_tool.setResultsDir(".\\")

## Remove absent markers, set weighting for bony landmarks

In [11]:
# List of bony anatomical landmarkers to give high weighting
bony_landmarks = ['LMMAL','RMMAL','LLMAL','RLMAL','LASI','RASI','LPSI','RPSI']

for marker in IK_tool.getIKTaskSet():
    # markers.getName() is the name of each marker
    current_marker = marker.getName()

    # If the marker in the inverse kinematics tool is one which we are using
    if (current_marker in marker_names):
        marker.setApply(True)
    else:
        marker.setApply(False)

    if current_marker in bony_landmarks:
        marker.setWeight(10)
    else:
        marker.setWeight(1)

## Write to an XML setup file

In [12]:
xml_setup_path = directory + "\\" + model + "\\" + trial + "\\" + trial + ik_filename.split("\\")[-1]
IK_tool.printToXML(xml_setup_path)

''' Temporary fix for setting model name using XML parsing '''

dom = minidom.parse(xml_setup_path)
dom.getElementsByTagName("model_file")[0].firstChild.nodeValue = model + ".osim"

with open(xml_setup_path, 'w') as xml_file:
    dom.writexml(xml_file, addindent='\t', newl='\n', encoding='UTF-8')

# Using minidom will often create unneeded white space
xml_shorten(xml_setup_path)